In [1]:
import duckdb
conn = duckdb.connect("week3/database.db")
cur = conn.cursor()

## Cursors
Cursors are an object that mediates between submitting a query and getting the desired results. If we wanted to run several queries, we'd want more cursors. Cursors return a list.

__Example:__ Return all values

In [ ]:
cur.fetchall()

__Example:__ Execute a select statement

In [7]:
cur.execute("SELECT Nest_ID FROM Bird_nests LIMIT 10")
[t[0] for t in cur.fetchall()]

['14HPE1',
 '11eaba',
 '11eabaagc01',
 '11eabaagv01',
 '11eababbc02',
 '11eababsv01',
 '11eabaduh01',
 '11eabaduv01',
 '11eabarpc01',
 '11eabarpc02']

__Example:__ Return All or Single Value

In [8]:
# return all values
cur.execute("SELECT COUNT(*) FROM Bird_nests")
print(cur.fetchall())

# return single value
cur.execute("SELECT COUNT(*) FROM Bird_nests")
cur.fetchone()

[(1547,)]


(1547,)

__Example:__ Using an iterator
This will not run because duckdb does not support iterators

NOTE: Avoid using `SELECT * FROM` since you are relying on the selection of rows. To create a more "stable"

__Example:__ How many nests do we have for each species?

Approach 1: First get all species, then execute a count query for each species.

In [13]:
# create a general query, where %s will be filled in with the sp code
query = " SELECT COUNT (*) FROM Bird_nests WHERE Species = '%s' "

# apply query accross all species
cur.execute("SELECT Code FROM Species LIMIT 3")
for row in cur.fetchall():
    code = row[0]
    prepared_query = query % code
    #print(prepared_query)
    cur2 = conn.cursor()
    cur2.execute(prepared_query)
    print(f"Species {code} has {cur2.fetchone()[0]} nests")
    cur2.close()

Species agsq has 0 nests
Species amcr has 0 nests
Species amgp has 29 nests


Approach 2: Instead of using %s, use a ? in the query. Note the `cur2.execute()` information changes, and this code is slightly shorter.

In [16]:
# create a general query using ? instead of %s
query = " SELECT COUNT (*) FROM Bird_nests WHERE Species = ? "

# apply query accross all species
cur.execute("SELECT Code FROM Species LIMIT 3")
for row in cur.fetchall():
    code = row[0]
    cur2 = conn.cursor()
    cur2.execute(query, [code])
    print(f"Species {code} has {cur2.fetchone()[0]} nests")
    cur2.close()

Species agsq has 0 nests
Species amcr has 0 nests
Species amgp has 29 nests


There are some dangers in using the string method, seen in Approach 1.

__Example:__

In [22]:
# add a new row to the Personnel table using the %s syntax
abbrev = "TS"
name = "Taylor Swift"
cur.execute("INSERT INTO Personnel (Abbreviation, Name) VALUES ('%s', '%s')" % (abbrev, name))

# see what this row looks like
cur.execute()
cur.fetchall()

ConstraintException: Constraint Error: Duplicate key "Abbreviation: TS" violates primary key constraint. If this is an unexpected constraint violation please double check with the known index limitations section in our documentation (https://duckdb.org/docs/sql/indexes).